In [ ]:
def evaluate_model_cv_lgbm(model, X, y, n_folds=5):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=RANDOM_STATE)
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='auc',
        )

        y_pred = model.predict_proba(X_val)[:, 1]
        scores.append(roc_auc_score(y_val, y_pred))

    return np.mean(scores), np.std(scores)


lgbm_cv_score, lgbm_cv_std = evaluate_model_cv_lgbm(lgbm_model, X_train, y_train)

In [ ]:
def evaluate_model_cv_catboost(model, X, y, n_folds=5):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=RANDOM_STATE)
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
        )

        y_pred = model.predict_proba(X_val)[:, 1]
        scores.append(roc_auc_score(y_val, y_pred))

    return np.mean(scores), np.std(scores)


catboost_cv_score, catboost_cv_std = evaluate_model_cv_catboost(catboost_model, X_train, y_train)

In [ ]:
50
iters
optuna

lgbm_best_params = {'num_leaves': 175, 'learning_rate': 0.010203959463015153, 'max_depth': 11, 'min_child_samples': 29,
                    'subsample': 0.9595377318249911, 'colsample_bytree': 0.7008023302141809,
                    'reg_alpha': 0.7369604631780851,
                    'reg_lambda': 0.6668252470437909, 'n_estimators': 3000,
                    'objective': 'binary',
                    'boosting_type': 'gbdt',
                    'metric': 'auc',
                    'n_jobs': -1,
                    'random_state': 42,
                    'verbose': -1, }

In [ ]:
catboost_best_params = {'iterations': 1661, 'learning_rate': 0.028239830074159606, 'depth': 10, 'eval_metric': 'AUC',
                        'random_seed': RANDOM_SEED,
                        'verbose': False,
                        'cat_features': df_train_new.select_dtypes(exclude='number').columns.to_list(),
                        'task_type': 'GPU',
                        'devices': '0', }

In [ ]:
rf_best_params = {
    'n_estimators': 441, 'max_depth': 26, 'max_features': 'sqrt',
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbose': 0,
}

In [ ]:
first_model = predictor.model_names()[0]  # Берем первую модель
test_proba_first_model = predictor.predict_proba(df_test_new, model=first_model)

if isinstance(test_proba_first_model, pd.DataFrame):
    test_predictions = test_proba_first_model.iloc[:, 1]  # Вероятность класса 1
else:
    test_predictions = test_proba_first_model

pd.DataFrame({
    'TransactionID': sample_submission['TransactionID'],
    'isFraud': test_predictions
}).to_csv('autogluon_submission.csv', index=False)